In [3]:
import pandas as pd
import geopandas as gpd
import osmnx
import sgeop
import momepy
import json
import geoplanar

## 2D Buildings

Building footprints from the ALKIS cadastre data https://opengeodata.lgl-bw.de/#/(sidenav:product/1) were downloaded for the following regions.

In [1]:

paths = [
    "ALKIS-oE_085280_Bötzingen_shp/",
    "ALKIS-oE_085300_Umkirch_shp/",
    "ALKIS-oE_085714_Lehen_shp/",
    "ALKIS-oE_085292_Hugstetten_shp/",
    "ALKIS-oE_085290_Buchheim_shp/",
    "ALKIS-oE_085293_Neuershausen_shp/",
    "ALKIS-oE_085291_Holzhausen_shp/",
    "ALKIS-oE_085712_Hochdorf_shp/",
    "ALKIS-oE_085240_Vörstetten_shp/",
    "ALKIS-oE_085235_Denzlingen_shp/",
    "ALKIS-oE_085350_Föhrental_shp/",
    "ALKIS-oE_085335_Heuweiler_shp/",
    "ALKIS-oE_085341_Wildtal_shp/",
    "ALKIS-oE_085340_Gundelfingen_shp/",
    "ALKIS-oE_085420_Stegen_shp/",
    "ALKIS-oE_085422_Wittental_shp/",
    "ALKIS-oE_085711_Ebnet_shp/",
    "ALKIS-oE_085432_Zarten_shp/",
    "ALKIS-oE_085430_Kirchzarten_shp/",
    "ALKIS-oE_085713_Kappel_shp/",
    "ALKIS-oE_085710_Freiburg_shp/",
    "ALKIS-oE_085415_Horben_shp/",
    "ALKIS-oE_085410_Sölden_shp/",
    "ALKIS-oE_085405_Wittnau_shp/",
    "ALKIS-oE_085400_Au_shp/",
    "ALKIS-oE_085390_Merzhausen_shp/",
    "ALKIS-oE_085380_Ebringen_shp/",
    "ALKIS-oE_085372_Wolfenweiler_shp/",
    "ALKIS-oE_085385_Pfaffenweiler_shp/",
    "ALKIS-oE_085512_Norsingen_shp/",
    "ALKIS-oE_085514_Scherzingen_shp/",
    "ALKIS-oE_085371_Schallstadt_shp/",
    "ALKIS-oE_085370_Mengen_shp/",
    "ALKIS-oE_085715_Munzingen_shp/",
    "ALKIS-oE_085717_Tiengen_shp/",
    "ALKIS-oE_085716_Opfingen_shp/",
    "ALKIS-oE_085718_Waltershofen_shp/",
    "ALKIS-oE_085330_Merdingen_shp/",
    "ALKIS-oE_085285_Gottenheim_shp/",
    "ALKIS-oE_085311_Wasenweiler_shp/"
]

In [4]:
building_gdfs = []

In [5]:
for i in paths:
    print(i)
    buildings = gpd.read_file('../data/raw_data/' + i + 'gebaeudeBauwerke.shp')
    buildings = buildings[buildings['gebnutzbez'] == "Gebaeude"]
    buildings = buildings[~buildings['funktion'].isin(['Tiefgarage','Schuppen','Gartenhaus','Schutzhütte','Stall','Wasserbehälter','Scheune und Stall','Garage','Scheune','Gebäude für Vorratshaltung','Umformer'])]
    buildings = buildings.to_crs(31468)
    building_gdfs.append(buildings)

ALKIS-oE_085280_Bötzingen_shp/
ALKIS-oE_085300_Umkirch_shp/
ALKIS-oE_085714_Lehen_shp/
ALKIS-oE_085292_Hugstetten_shp/
ALKIS-oE_085290_Buchheim_shp/
ALKIS-oE_085293_Neuershausen_shp/
ALKIS-oE_085291_Holzhausen_shp/
ALKIS-oE_085712_Hochdorf_shp/
ALKIS-oE_085240_Vörstetten_shp/
ALKIS-oE_085235_Denzlingen_shp/
ALKIS-oE_085350_Föhrental_shp/
ALKIS-oE_085335_Heuweiler_shp/
ALKIS-oE_085341_Wildtal_shp/
ALKIS-oE_085340_Gundelfingen_shp/
ALKIS-oE_085420_Stegen_shp/
ALKIS-oE_085422_Wittental_shp/
ALKIS-oE_085711_Ebnet_shp/
ALKIS-oE_085432_Zarten_shp/
ALKIS-oE_085430_Kirchzarten_shp/
ALKIS-oE_085713_Kappel_shp/
ALKIS-oE_085710_Freiburg_shp/
ALKIS-oE_085415_Horben_shp/
ALKIS-oE_085410_Sölden_shp/
ALKIS-oE_085405_Wittnau_shp/
ALKIS-oE_085400_Au_shp/
ALKIS-oE_085390_Merzhausen_shp/
ALKIS-oE_085380_Ebringen_shp/
ALKIS-oE_085372_Wolfenweiler_shp/
ALKIS-oE_085385_Pfaffenweiler_shp/
ALKIS-oE_085512_Norsingen_shp/
ALKIS-oE_085514_Scherzingen_shp/
ALKIS-oE_085371_Schallstadt_shp/
ALKIS-oE_085370_Meng

In [6]:
all_buildings = pd.concat(building_gdfs, ignore_index=True)

In [7]:
all_buildings.to_parquet('../data/processed_data/preprocessed_buildings_unfiltered.parquet')

In [19]:
# find bounding box of data
bounds = all_buildings.to_crs('epsg:4326').total_bounds

## 3D Buildings

Merge with 3D LoD1 CityJSON data from https://geodaten.freiburg.de/geonetwork/srv/ger/catalog.search#/metadata/164ceef4-6c67-4763-8100-c5799a2ea6d8 (converted from CityGML using https://www.cityjson.org/tutorials/conversion/ and analysed using https://github.com/tudelft3d/3d-building-metrics) to obtain the heights and the 3d form statistics

In [20]:
def read_json_file(file_name):
    with open(file_name) as f:
        data = json.load(f)
    return data

In [21]:
data = read_json_file('../data/raw_data/2023_Freiburg_LoD1.json')
# read data into dataframe
df = pd.DataFrame.from_dict(data['CityObjects'], orient='index')
df[['Bodenhoehe', 'Firsthoehe', 'Traufhoehe', 'measuredHeight', 'roofType','Gemeindeschluessel', 'Hausnummer', 'Lagebezeichnung', 'class', 'name']] = df['attributes'].apply(pd.Series)
df[df['geometry'].apply(pd.Series).columns] = df['geometry'].apply(pd.Series)

In [22]:
df

,type,attributes,geometry,Bodenhoehe,Firsthoehe,Traufhoehe,measuredHeight,roofType,Gemeindeschluessel,Hausnummer,Lagebezeichnung,class,name,0
DEBWL0010007TIIT,Building,"{'Bodenhoehe': '254.218', 'Firsthoehe': '272.8...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",254.218,272.838,272.838,18.620,1000,NaN,NaN,NaN,NaN,NaN,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."
DEBWL0010007TIIU,Building,"{'Bodenhoehe': '254.520', 'Firsthoehe': '257.5...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",254.520,257.504,257.504,2.984,1000,NaN,NaN,NaN,NaN,NaN,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."
DEBWL0010007TII4,Building,"{'Bodenhoehe': '250.677', 'Firsthoehe': '257.9...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",250.677,257.946,257.946,7.269,1000,NaN,NaN,NaN,NaN,NaN,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."
DEBWL001000i26bW,Building,"{'Bodenhoehe': '239.074', 'Firsthoehe': '241.6...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",239.074,241.617,241.617,2.543,1000,NaN,NaN,NaN,NaN,NaN,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."
DEBWL0010007TIHZ,Building,"{'Bodenhoehe': '250.696', 'Firsthoehe': '256.8...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",250.696,256.876,256.876,6.181,1000,NaN,NaN,NaN,NaN,NaN,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DEBWL001000g414h,Building,"{'name': 'Zedernhütte', 'Bodenhoehe': '315.967...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",315.967,333.999,333.999,18.032,1000,08311000,NaN,Distr. Illenberg,Öffentlich,Zedernhütte,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."
DEBWL001000s0iiB,Building,"{'Bodenhoehe': '301.605', 'Firsthoehe': '305.2...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",301.605,305.280,305.280,3.675,1000,08311000,8,Wonnhaldestraße,Öffentlich,NaN,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."
DEBWL0010007TAhF,Building,"{'Bodenhoehe': '306.829', 'Firsthoehe': '312.9...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",306.829,312.915,312.915,6.086,1000,08311000,NaN,Wonnhaldestraße,Sonstige,NaN,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."
DEBWL0010007TAgD,Building,"{'Bodenhoehe': '369.483', 'Firsthoehe': '372.5...","[{'type': 'Solid', 'lod': '1', 'boundaries': [...",369.483,372.509,372.509,3.026,1000,08311000,NaN,Schauinslandstraße,Sonstige,NaN,"{'type': 'Solid', 'lod': '1', 'boundaries': [[..."


In [23]:
#check if all building oid end with BL
all_buildings["oid"].str.endswith("BL").all()

True

In [24]:
#remove BL from oid
all_buildings["oid"] = all_buildings["oid"].str[:-2]

In [25]:
all_buildings = all_buildings.merge(df,left_on='oid',right_on=df.index,how='left', suffixes=('_2d', '_3d'))
all_buildings = all_buildings.set_geometry('geometry_2d')

In [29]:
all_buildings['measuredHeight'].value_counts()

measuredHeight
10.000    128
3.000      24
8.738      16
8.475      14
8.293      14
         ... 
12.950      1
2.697       1
15.692      1
4.329       1
4.519       1
Name: count, Length: 12967, dtype: int64

In [ ]:
all_buildings[['oid','geometry_2d']].explore()

## Street data

Extract street network from Overture for the region with building footprints and simplify using https://github.com/uscuni/sgeop

In [18]:
bounds[3],bounds[1],bounds[2],bounds[0]

(48.0922300969916, 47.90948492578161, 7.998250541041024, 7.669916729552059)

In [19]:
!overturemaps download --bbox=7.669916729552059,47.90948492578161,7.998250541041024,48.0922300969916 -f geoparquet --type=segment -o fbg_streets_overture.parquet

In [20]:
streets = gpd.read_parquet('../data/raw_data/fbg_streets_overture.parquet')

In [21]:
## service road removed
approved_roads = ['living_street',
                    'motorway',
                    'motorway_link',
                    'pedestrian',
                    'primary',
                    'primary_link',
                    'residential',
                    'secondary',
                    'secondary_link',
                    'tertiary',
                    'tertiary_link',
                    'trunk',
                    'trunk_link',
                    'unclassified']
streets = streets[streets['class'].isin(approved_roads)]

In [22]:
streets = streets.to_crs(epsg=31468)

In [23]:
streets = sgeop.simplify_network(streets)

/Users/lisawink/.conda/envs/py312_uscuni_buildingfootprints/lib/python3.12/site-packages/sgeop/simplify.py:377: UserWarning: Could not create a connection as it would lead outside of the artifact.
  nx_gx_cluster(


In [ ]:
streets.explore()

In [31]:
all_buildings["street_index"] = momepy.get_nearest_street(all_buildings, streets)

In [32]:
all_buildings.columns = all_buildings.columns.astype(str)

In [27]:
streets.to_parquet('../data/processed_data/preprocessed_streets.parquet')

In [33]:
all_buildings.to_parquet('../data/processed_data/preprocessed_buildings.parquet')